In [8]:
#https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks/tree/master/Python

#https://simpleitk.org/doxygen/latest/html/examples.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethod1_2ImageRegistrationMethod1_8py-example.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethod2_2ImageRegistrationMethod2_8py-example.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethod3_2ImageRegistrationMethod3_8py-example.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethod4_2ImageRegistrationMethod4_8py-example.html

#B_spline registration
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethodBSpline1_2ImageRegistrationMethodBSpline1_8py-example.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethodBSpline2_2ImageRegistrationMethodBSpline2_8py-example.html
#https://simpleitk.org/doxygen/latest/html/ImageRegistrationMethodBSpline3_2ImageRegistrationMethodBSpline3_8py-example.html
#http://simpleitk.org/SimpleITK-Notebooks/01_Image_Basics.html

import numpy as np
import glob
import pydicom
import os
from pycimg import CImg
import nibabel as nib
from scipy.interpolate import interpn
from skimage.filters import gaussian
import SimpleITK as sitk
import sys

In [9]:
import platform

In [10]:
im1Name = './Data/T1_image_0007.nii.gz'
im2Name = './Data/DWI_image_0006.nii.gz'

In [11]:
def command_iteration(method):
     print(f"{method.GetOptimizerIteration():3} = {method.GetMetricValue():10.5f} : {method.GetOptimizerPosition()}")
 

In [12]:
fixed = sitk.ReadImage(im1Name, sitk.sitkFloat32)   # to jest segmentacja 
fixed = sitk.DiscreteGaussian(fixed, 5.0)           # więc ją nieco rozmywam

moving = sitk.ReadImage(im2Name, sitk.sitkFloat32)  # to też jest segmentacja 
moving = sitk.DiscreteGaussian(moving, 5.0)         # więc ją nieco rozmywam


In [158]:
# Najpierw znajduję transformację afiniczną moving->fixed

R = sitk.ImageRegistrationMethod()

R.SetMetricAsCorrelation()
R.SetOptimizerAsRegularStepGradientDescent(learningRate=2.1,
                                        minStep=1e-5,
                                        numberOfIterations=125,
                                        gradientMagnitudeTolerance=1e-8)
R.SetOptimizerScalesFromIndexShift()
tx = sitk.CenteredTransformInitializer(fixed, moving,
                                    sitk.Similarity3DTransform())
R.SetInitialTransform(tx)
R.SetInterpolator(sitk.sitkLinear)

# to można odkomentować, żeby widzieć postęp
R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

# Aplikuję transformację do "moving image"

print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

#sitk.WriteTransform(outTx, 'transform.txt')
#print(outTx)
  
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(0)
resampler.SetTransform(outTx)

# to wynik transformacji - obraz SimpleITK moving nałożony na obraz fixed
out = resampler.Execute(moving)


  0 =   -0.89814 : (0.0006742567231165365, -0.00118748156934228, 0.0016762156431556429, -14.66885739912062, -0.4677592229303103, 23.590292738731577, 1.0008577306733661)
  1 =   -0.90623 : (0.0008342283554663808, -0.0010903347973603584, 0.0019812514484419588, -14.699788488631807, -0.6063307551767507, 22.575174191687914, 1.0010036695361662)
  2 =   -0.91061 : (0.001303884634322579, -0.000991153555238188, 0.00274743473259, -14.586181074919553, -0.8275992846610568, 21.580812498311488, 1.0013862034679635)
  3 =   -0.90925 : (0.0018755121374901026, -0.000926045445743971, 0.003484126454260302, -14.36987178249547, -0.918881682110039, 22.036367565491275, 1.0017732525574456)
  4 =   -0.91112 : (0.0022222121523622447, -0.0007562598485514545, 0.003894896322786571, -14.360934965997881, -0.9929303750332761, 21.791215226325328, 1.001988732454324)
  5 =   -0.91156 : (0.0037364259315455584, -8.2143910719426e-05, 0.005600040765621031, -14.237204202777669, -1.19914794740639, 21.70286572693301, 1.00289412

In [159]:
# A teraz znajduję transformację odwrotną
inverse_transform = outTx.GetInverse()

# i przekształcam wynik transformacji outTx transformacją do niej odwrotną - w teorii powinno to być przekształcenie identycznościowe
movingFromInverse = sitk.Resample(out,moving,inverse_transform,sitk.sitkLinear,0,out.GetPixelID())


In [160]:
# przerabiam obrazki SimpleITK na macierze numpy
fixedNP = sitk.GetArrayFromImage(fixed)
movingNP = sitk.GetArrayFromImage(moving)
outNP = sitk.GetArrayFromImage(out)
movingFromInverseNP = sitk.GetArrayFromImage(movingFromInverse)

# i sprawdzam, jak zadziałało przekształcenie "identycznościowe"
print(np.sum(np.abs(movingNP-outNP)),np.sum(np.abs(movingNP-movingFromInverseNP)))

if platform.system() != "Darwin":
    CImg(fixedNP-outNP).display();
    CImg(movingNP-outNP).display();
    CImg(movingNP-movingFromInverseNP).display();

235050.94 6598.609


In [161]:
#metryki jakości nakładania

from scipy.stats import spearmanr
from scipy.stats import pearsonr
import skimage

array_a = np.ndarray.flatten(fixedNP)
array_b = np.ndarray.flatten(movingNP)
array_c = np.ndarray.flatten(outNP)

print(pearsonr(array_a, array_b)[0],spearmanr(array_a, array_b).correlation,skimage.metrics.mean_squared_error(array_a, array_b),skimage.metrics.normalized_root_mse(array_a, array_b),skimage.metrics.structural_similarity(array_a, array_b, data_range=1))
print(pearsonr(array_a, array_c)[0],spearmanr(array_a, array_c).correlation,skimage.metrics.mean_squared_error(array_a, array_c),skimage.metrics.normalized_root_mse(array_a, array_c),skimage.metrics.structural_similarity(array_a, array_c, data_range=1))

0.7884047908594654 0.801846208271481 0.028753029132587177 0.63050174400885 0.9247642197688857
0.9654193657114964 0.9484777546437263 0.004605925140480957 0.2523497783365449 0.9676938587903938


In [162]:
# W kolejnym kroku znajduję transformacje elastyczną outTx(moving)->fixed

transformDomainMeshSize = [2] * moving.GetDimension()
tx1 = sitk.BSplineTransformInitializer(fixed,transformDomainMeshSize)

print("Initial Parameters:")
print(tx.GetParameters())

R1 = sitk.ImageRegistrationMethod()
R1.SetMetricAsCorrelation()

R1.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5,
                    numberOfIterations=10,
                    maximumNumberOfCorrections=5,
                    maximumNumberOfFunctionEvaluations=1000,
                    costFunctionConvergenceFactor=1e+7)
R1.SetInitialTransform(tx1, True)
R1.SetInterpolator(sitk.sitkLinear)

# to można odkomentować, żeby widzieć postęp
#R1.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R1))

outTx1 = R1.Execute(fixed, out)

# Aplikuję transformację do outTx(moving)

print("-------")
print(outTx1)
print(f"Optimizer stop condition: {R1.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R1.GetOptimizerIteration()}")
print(f" Metric value: {R1.GetMetricValue()}")

  
resampler1 = sitk.ResampleImageFilter()
resampler1.SetReferenceImage(fixed)
resampler1.SetInterpolator(sitk.sitkLinear)
resampler1.SetDefaultPixelValue(0)
resampler1.SetTransform(outTx1)

out1 = resampler1.Execute(out)


Initial Parameters:
(0.02173480841060934, 0.013845642738657843, 0.028444543402080726, -15.363586328858087, -0.7746600254548354, 22.002504162692915, 1.0122057304402527)
-------
itk::simple::BSplineTransform
 BSplineTransform (0x7fd74a438740)
   RTTI typeinfo:   itk::BSplineTransform<double, 3u, 3u>
   Reference Count: 3
   Modified Time: 41077
   Debug: Off
   Object Name: 
   Observers: 
     none
   CoefficientImage: [ 0x7fd72a025730, 0x7fd72a0259e0, 0x7fd72a025c90 ]
   TransformDomainOrigin: [-313.281, -398.281, -1.875]
   TransformDomainPhysicalDimensions: [314.062, 399.062, 243.75]
   TransformDomainDirection: 1 0 0
0 1 0
0 0 1

   TransformDomainMeshSize: [2, 2, 2]
   GridSize: [5, 5, 5]
   GridOrigin: [-470.312, -597.812, -123.75]
   GridSpacing: [157.031, 199.531, 121.875]
   GridDirection: 1 0 0
0 1 0
0 0 1


Optimizer stop condition: LBFGSBOptimizerv4: User requested
 Iteration: 10
 Metric value: -0.9535588383493665


In [163]:
# potrzebna jest transformacja odwrotna, niestety metoda GetInverse() dla Similarity3DTransform nie działa dla BSplineTransform
# i ponizsza linia generuje błąd
try:
    inverse_transform1 = outTx1.GetInverse()
except:
    print('GetInverse does not work')


GetInverse does not work


In [164]:
# Wizualizacja i miary jakości nakładania

outNP1 = sitk.GetArrayFromImage(out1)
array_d = np.ndarray.flatten(outNP1)

print(pearsonr(array_a, array_c)[0],spearmanr(array_a, array_c).correlation,skimage.metrics.mean_squared_error(array_a, array_c),skimage.metrics.normalized_root_mse(array_a, array_c),skimage.metrics.structural_similarity(array_a, array_c))
print(pearsonr(array_a, array_d)[0],spearmanr(array_a, array_d).correlation,skimage.metrics.mean_squared_error(array_a, array_d),skimage.metrics.normalized_root_mse(array_a, array_d),skimage.metrics.structural_similarity(array_a, array_d))
print(np.sum(np.abs(fixedNP-movingNP)),np.sum(np.abs(fixedNP-outNP)),np.sum(np.abs(fixedNP-outNP1)))

if platform.system() != "Darwin":
    CImg(fixedNP-outNP).display();
    CImg(fixedNP-outNP1).display();


ValueError: Since image dtype is floating point, you must specify the data_range parameter. Please read the documentation carefully (including the note). It is recommended that you always specify the data_range anyway.

In [ ]:
#zapiszmy transformacje 
sitk.WriteTransform(outTx, 'transform.txt')
sitk.WriteTransform(outTx1, 'transform1.txt')

#przeczytajmy transformacje 
t1 = sitk.ReadTransform('transform.txt')
t2 = sitk.ReadTransform('transform1.txt')

#zaaplikujmy transformacje do obrazów
im1 = sitk.Resample(moving,fixed,t1,sitk.sitkLinear,0,fixed.GetPixelID())
im2 = sitk.Resample(im1,fixed,t2,sitk.sitkLinear,0,fixed.GetPixelID())

#porównajmy wyniki działania transformacji wczytanych z plików  z wyliczeniami z komórek powyżej - obraz foo powinien być identyczny z obrazem outNP1
foo = sitk.GetArrayFromImage(im2)

if platform.system() != "Darwin":
    CImg(outNP1 - foo).display();